In [1]:
from sklearn import preprocessing
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import sys
import os
sys.path.append(os.path.abspath('../src'))
import data_prep, ai_config
import importlib
importlib.reload(data_prep)

<module 'data_prep' from '/Users/wihill/git/AI_Based_Hyperparameter_Tuning/src/data_prep.py'>

# LOAD RAW

In [2]:
%%time
# 45 seconds
print(datetime.datetime.now())

df = data_prep.get_ieee_fraud_data()

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')   

2020-02-10 20:39:50.150410
START train.csv
train - mean fraud 0.03499000914417313
END train.csv
-0.4429586841786278


(1097231, 433)

3.55 

float64    399
object      31
int64        3
dtype: int64 

CPU times: user 32.7 s, sys: 8.15 s, total: 40.8 s
Wall time: 41.1 s


In [4]:
%%time
# 11 SECONDS
df.reset_index().to_feather('raw.ftr')

CPU times: user 8.47 s, sys: 14.1 s, total: 22.6 s
Wall time: 14.8 s


In [33]:
%%time
# 10 SECONDS

df = pd.read_feather('raw.ftr').set_index('TransactionID')

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')   

(1097231, 433)

3.55 

float64    399
object      31
int64        3
dtype: int64 

CPU times: user 4.79 s, sys: 12 s, total: 16.8 s
Wall time: 11.5 s


In [37]:
df

,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,devicetype,deviceinfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,credit,...,None,NaN,None,None,None,None,None,None,None,None
2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,credit,...,None,NaN,None,None,None,None,None,None,None,None
2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,debit,...,None,NaN,None,None,None,None,None,None,None,None
2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,debit,...,None,NaN,None,None,None,None,None,None,None,None
2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170235,-1,34214279,94.679,C,13832,375.0,185.0,mastercard,224.0,debit,...,None,NaN,None,None,None,None,None,None,None,None
4170236,-1,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,debit,...,chrome 43.0 for android,NaN,None,None,F,F,T,F,mobile,ALE-L23 Build/HuaweiALE-L23
4170237,-1,34214326,49.000,W,16661,490.0,150.0,visa,226.0,debit,...,None,NaN,None,None,None,None,None,None,None,None


In [38]:
df.sort_values(by=['transactiondt'])

,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,devicetype,deviceinfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,credit,...,None,NaN,None,None,None,None,None,None,None,None
2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,credit,...,None,NaN,None,None,None,None,None,None,None,None
2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,debit,...,None,NaN,None,None,None,None,None,None,None,None
2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,debit,...,None,NaN,None,None,None,None,None,None,None,None
2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170235,-1,34214279,94.679,C,13832,375.0,185.0,mastercard,224.0,debit,...,None,NaN,None,None,None,None,None,None,None,None
4170236,-1,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,debit,...,chrome 43.0 for android,NaN,None,None,F,F,T,F,mobile,ALE-L23 Build/HuaweiALE-L23
4170237,-1,34214326,49.000,W,16661,490.0,150.0,visa,226.0,debit,...,None,NaN,None,None,None,None,None,None,None,None


# OBJECT TO NUMERIC

In [6]:
%%time
# 3 SECONDS
print(datetime.datetime.now())

for c in tqdm(df.columns):
    if df[c].dtype == 'object':
        df[c] = pd.to_numeric(df[c], errors='ignore')

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')  

2020-02-10 20:41:12.249772


(1097231, 433)

3.55 

float64    399
object      31
int64        3
dtype: int64 

CPU times: user 720 ms, sys: 31 ms, total: 751 ms
Wall time: 748 ms


In [7]:
%%time
# 7 SECONDS
df.reset_index().to_feather('obj2num.ftr')

CPU times: user 5.43 s, sys: 3.44 s, total: 8.87 s
Wall time: 6.87 s


In [8]:
%%time
# 7 SECONDS

df = pd.read_feather('obj2num.ftr').set_index('TransactionID')

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')   

(1097231, 433)

3.55 

float64    399
object      31
int64        3
dtype: int64 

CPU times: user 4.24 s, sys: 5.19 s, total: 9.43 s
Wall time: 6.14 s


# IMPUTE

In [9]:
%%time
# 13 SECONDS

for c in tqdm(df.columns):

    if df[c].isnull().sum() > 0:

        if np.issubdtype(df[c].dtype, np.number):
            if ai_config.NUM_IMPUTE_TYPE == 'MEDIAN':
                df[c] = df[c].fillna(df[c].median())
        else:
            df[c] = df[c].fillna(df[c].mode()[0])
            
display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts())        

(1097231, 433)

3.55 

float64    399
object      31
int64        3
dtype: int64
CPU times: user 11.8 s, sys: 1.3 s, total: 13.1 s
Wall time: 13.1 s


# CHECK FOR MISSING

In [10]:
for c in tqdm(df.columns.tolist()):
    if df[c].isnull().sum() > 0:
        print(c)

In [11]:
%%time
# 6 SECONDS
df.reset_index().to_feather('imputed.ftr')

CPU times: user 5.12 s, sys: 2.71 s, total: 7.83 s
Wall time: 5.94 s


In [12]:
%%time
# 7 SECONDS

df = pd.read_feather('imputed.ftr').set_index('TransactionID')

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n') 

(1097231, 433)

3.55 

float64    399
object      31
int64        3
dtype: int64 

CPU times: user 3.7 s, sys: 4.16 s, total: 7.87 s
Wall time: 5.95 s


# FLOAT > INT

In [14]:
%%time
# 7 MINUTES
print(datetime.datetime.now())

pbar = tqdm(df.columns)
for c in pbar:        
    pbar.set_description("Processing %s" % c)
    if str(df[c].dtypes)[:5] == 'float':
        # print('column ', c)        
        df[c] = pd.to_numeric(df[c], errors='ignore', downcast='integer')


display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts())        

2020-02-10 20:42:20.708467


(1097231, 433)

1.37 

int8       228
int16       90
float64     81
object      31
int64        3
dtype: int64
CPU times: user 3min 46s, sys: 2min 57s, total: 6min 43s
Wall time: 6min 43s


In [15]:
%%time
# 4 SECONDS
df.reset_index().to_feather('flt_2_int.ftr')

CPU times: user 2.43 s, sys: 1.6 s, total: 4.03 s
Wall time: 3.7 s


In [16]:
%%time
# 2 SECONDS

df = pd.read_feather('flt_2_int.ftr').set_index('TransactionID')

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')    

(1097231, 433)

1.37 

int8       228
int16       90
float64     81
object      31
int64        3
dtype: int64 

CPU times: user 2.03 s, sys: 1.23 s, total: 3.26 s
Wall time: 2.04 s


# Minify

In [19]:
%%time
# 15 seconds
print(datetime.datetime.now())

data_prep.minify(df)

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')    

2020-02-10 21:31:25.782323
Mem. usage decreased to  1.02 GB (25.5% reduction)


(1097231, 433)

1.02 

int8       229
int16       91
float32     81
object      31
int32        1
dtype: int64 

CPU times: user 8.62 s, sys: 6.39 s, total: 15 s
Wall time: 15 s


In [20]:
%%time
# 4 SECONDS
df.reset_index().to_feather('minified.ftr')

CPU times: user 2.39 s, sys: 1.36 s, total: 3.74 s
Wall time: 3.4 s


In [21]:
%%time
# 2 SECONDS

df = pd.read_feather('minified.ftr').set_index('TransactionID')

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')  

(1097231, 433)

1.02 

int8       229
int16       91
float32     81
object      31
int32        1
dtype: int64 

CPU times: user 1.56 s, sys: 711 ms, total: 2.28 s
Wall time: 1.57 s


# ENCODING  

In [23]:
%%time
# 70 SECONDS
print(datetime.datetime.now())

pbar = tqdm(df.columns)

for c in pbar:        

    pbar.set_description('encoding %s' % c)

    
    if df[c].dtype == 'object' and df[c].nunique() <= ai_config.MAX_N_OHE:
        pbar.set_description('OHE %s' % c)
        df = pd.concat([df, pd.get_dummies(df[c], drop_first=True, prefix=c)], axis=1)
        df = df.drop([c], axis=1)

    elif df[c].dtype == 'object':
        pbar.set_description('LE %s' % c)
        df[c] = preprocessing.LabelEncoder().fit_transform(df[c])
        

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')  

2020-02-10 21:31:57.548447


(1097231, 445)

0.85 

int8       229
int16       91
float32     81
uint8       37
int64        6
int32        1
dtype: int64 

CPU times: user 40.1 s, sys: 24.8 s, total: 1min 4s
Wall time: 1min 4s


In [24]:
%%time
# 4 SECONDS
df.reset_index().to_feather('encoded.ftr')

CPU times: user 888 ms, sys: 756 ms, total: 1.64 s
Wall time: 1.13 s


In [25]:
%%time
# 2 SECONDS

df = pd.read_feather('encoded.ftr').set_index('TransactionID')

display(df.shape)
print(round((df.memory_usage().sum() / 1024**3),2), '\n')
print(df.dtypes.value_counts(),'\n')  

(1097231, 445)

0.85 

int8       229
int16       91
float32     81
uint8       37
int64        6
int32        1
dtype: int64 

CPU times: user 944 ms, sys: 1.54 s, total: 2.48 s
Wall time: 799 ms


# FOLDS

In [27]:
kaggle_val = df[(df[ai_config.Y_COL] == -1)]

In [28]:
kaggle_val.reset_index().to_feather('kaggle_val.ftr')

In [29]:
df = df[(df[ai_config.Y_COL] != -1)]

In [30]:
df

,isfraud,transactiondt,transactionamt,card1,card2,card3,card5,addr1,addr2,dist1,...,id_28_New,id_29_NotFound,id_34_match_status:0,id_34_match_status:1,id_34_match_status:2,id_35_T,id_36_T,id_37_T,id_38_T,devicetype_mobile
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500000,13926,361,150,142,315,87,19,...,0,0,0,0,1,1,0,1,0,0
2987001,0,86401,29.000000,2755,404,150,102,325,87,8,...,0,0,0,0,1,1,0,1,0,0
2987002,0,86469,59.000000,4663,490,150,166,330,87,287,...,0,0,0,0,1,1,0,1,0,0
2987003,0,86499,50.000000,18132,567,150,117,476,87,8,...,0,0,0,0,1,1,0,1,0,0
2987004,0,86506,50.000000,4497,514,150,102,420,87,8,...,1,1,0,0,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535,0,15811047,49.000000,6550,361,150,226,272,87,48,...,0,0,0,0,1,1,0,1,0,0
3577536,0,15811049,39.500000,10444,225,150,224,204,87,8,...,0,0,0,0,1,1,0,1,0,0
3577537,0,15811079,30.950001,12037,595,150,224,231,87,8,...,0,0,0,0,1,1,0,1,0,0


In [31]:
%%time
# 11 SECONDS
print(datetime.datetime.now())

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True)

i = 0
for train, test in kf.split(df):
    df.iloc[train].reset_index().to_feather('../folds/train_' + str(i) + '.ftr')
    df.iloc[test].reset_index().to_feather('../folds/test_' + str(i) + '.ftr')
    i += 1

CPU times: user 8.92 s, sys: 4.58 s, total: 13.5 s
Wall time: 10.9 s


In [32]:
df

,isfraud,transactiondt,transactionamt,card1,card2,card3,card5,addr1,addr2,dist1,...,id_28_New,id_29_NotFound,id_34_match_status:0,id_34_match_status:1,id_34_match_status:2,id_35_T,id_36_T,id_37_T,id_38_T,devicetype_mobile
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500000,13926,361,150,142,315,87,19,...,0,0,0,0,1,1,0,1,0,0
2987001,0,86401,29.000000,2755,404,150,102,325,87,8,...,0,0,0,0,1,1,0,1,0,0
2987002,0,86469,59.000000,4663,490,150,166,330,87,287,...,0,0,0,0,1,1,0,1,0,0
2987003,0,86499,50.000000,18132,567,150,117,476,87,8,...,0,0,0,0,1,1,0,1,0,0
2987004,0,86506,50.000000,4497,514,150,102,420,87,8,...,1,1,0,0,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535,0,15811047,49.000000,6550,361,150,226,272,87,48,...,0,0,0,0,1,1,0,1,0,0
3577536,0,15811049,39.500000,10444,225,150,224,204,87,8,...,0,0,0,0,1,1,0,1,0,0
3577537,0,15811079,30.950001,12037,595,150,224,231,87,8,...,0,0,0,0,1,1,0,1,0,0


In [59]:
trs = []
for i in tqdm(range(0,10)):
    trs.append(pd.read_feather('test_' + str(i) + '.ftr').set_index('transactionid'))
all_trs = pd.concat(trs, axis=0)    